# Lab 48: Cloud Incident Response Automation

## Overview

Build automated incident response workflows for cloud environments using infrastructure-as-code, serverless functions, and orchestration tools.

**Difficulty**: Advanced  
**Duration**: 120-150 minutes  
**Prerequisites**: Labs 19, 19b, 19c, understanding of IaC concepts

## Learning Objectives

By the end of this lab, you will be able to:
1. Design automated cloud IR workflows
2. Implement containment actions using serverless
3. Build evidence collection automation
4. Create detection-to-response pipelines
5. Orchestrate multi-cloud incident response

**Next:** Lab 49 (LLM Red Teaming)

In [ ]:
#@title Install dependencies (Colab only)
#@markdown Run this cell to install required packages in Colab

%pip install -q pandas numpy

In [ ]:
import json
import hashlib
import uuid
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
from dataclasses import dataclass, field
from enum import Enum

print("Libraries loaded successfully!")

## Part 1: Containment Automation

Implement automated containment actions for compromised cloud resources.

### IR Automation Architecture

```
Detection Sources        Orchestration         Response Actions
+----------------+      +-------------+      +-----------------+
| SIEM/XDR       |----->|             |----->| Isolate Instance|
+----------------+      |             |      +-----------------+
| GuardDuty/Cloud|----->| Orchestrator|----->| Revoke Creds    |
+----------------+      |             |      +-----------------+
| Custom Rules   |----->|             |----->| Snapshot Disk   |
+----------------+      +-------------+      +-----------------+
```

In [ ]:
class ActionStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"
    ROLLED_BACK = "rolled_back"


@dataclass
class IRAction:
    """Represents an incident response action."""
    action_id: str
    action_type: str
    target: str
    status: ActionStatus
    timestamp: datetime
    details: Dict = field(default_factory=dict)
    rollback_info: Dict = field(default_factory=dict)


class EC2Containment:
    """EC2 instance containment automation."""
    
    def __init__(self):
        self.actions_log = []
    
    def isolate_instance(self, instance_id: str, isolation_sg_id: str, reason: str) -> IRAction:
        """Isolate an EC2 instance by replacing security groups."""
        action_id = str(uuid.uuid4())[:8]
        
        # Simulate getting current instance details
        original_sgs = ['sg-original-1', 'sg-original-2']  # Simulated
        
        action = IRAction(
            action_id=action_id,
            action_type='isolate_ec2',
            target=instance_id,
            status=ActionStatus.COMPLETED,
            timestamp=datetime.now(),
            details={
                'reason': reason,
                'isolation_sg': isolation_sg_id,
                'tags_added': [
                    {'Key': 'IR_Isolated', 'Value': 'true'},
                    {'Key': 'IR_IsolationTime', 'Value': datetime.now().isoformat()},
                    {'Key': 'IR_OriginalSGs', 'Value': json.dumps(original_sgs)}
                ]
            },
            rollback_info={
                'original_security_groups': original_sgs
            }
        )
        
        self.actions_log.append(action)
        return action
    
    def create_forensic_snapshot(self, instance_id: str, case_id: str) -> IRAction:
        """Create forensic snapshots of all volumes attached to an instance."""
        action_id = str(uuid.uuid4())[:8]
        
        # Simulated volume discovery and snapshot creation
        volumes = [
            {'volume_id': 'vol-0001', 'device_name': '/dev/sda1'},
            {'volume_id': 'vol-0002', 'device_name': '/dev/sdb'}
        ]
        
        snapshots = []
        for vol in volumes:
            snapshot_id = f'snap-{uuid.uuid4().hex[:8]}'
            snapshots.append({
                'snapshot_id': snapshot_id,
                'volume_id': vol['volume_id'],
                'device_name': vol['device_name']
            })
        
        action = IRAction(
            action_id=action_id,
            action_type='create_snapshot',
            target=instance_id,
            status=ActionStatus.COMPLETED,
            timestamp=datetime.now(),
            details={
                'case_id': case_id,
                'snapshots': snapshots,
                'volumes_processed': len(volumes)
            }
        )
        
        self.actions_log.append(action)
        return action

print("EC2 Containment ready!")

In [ ]:
# Demo: EC2 containment
print("EC2 CONTAINMENT DEMO")
print("=" * 50)

containment = EC2Containment()

# Isolate a compromised instance
print("\n1. Isolating compromised instance...")
isolate_action = containment.isolate_instance(
    instance_id='i-0123456789abcdef0',
    isolation_sg_id='sg-isolation-001',
    reason='GuardDuty alert: Cryptocurrency mining detected'
)

print(f"   Action ID: {isolate_action.action_id}")
print(f"   Status: {isolate_action.status.value}")
print(f"   Original SGs stored: {isolate_action.rollback_info['original_security_groups']}")

# Create forensic snapshots
print("\n2. Creating forensic snapshots...")
snapshot_action = containment.create_forensic_snapshot(
    instance_id='i-0123456789abcdef0',
    case_id='CASE-2024-001'
)

print(f"   Action ID: {snapshot_action.action_id}")
print(f"   Status: {snapshot_action.status.value}")
print(f"   Snapshots created: {len(snapshot_action.details['snapshots'])}")

for snap in snapshot_action.details['snapshots']:
    print(f"     - {snap['snapshot_id']} ({snap['device_name']})")

## Part 2: IAM Credential Revocation

Automate IAM credential revocation for compromised accounts.

In [ ]:
class IAMContainment:
    """IAM credential revocation automation."""
    
    def __init__(self):
        self.actions_log = []
    
    def revoke_user_credentials(self, user_name: str, access_key_id: str = None) -> IRAction:
        """Revoke IAM user credentials."""
        action_id = str(uuid.uuid4())[:8]
        actions_taken = []
        
        # Simulate disabling access keys
        if access_key_id:
            actions_taken.append(f'Disabled access key {access_key_id}')
        else:
            # Simulate listing and disabling all keys
            keys = ['AKIA1234567890ABCDEF', 'AKIA0987654321FEDCBA']
            for key in keys:
                actions_taken.append(f'Disabled access key {key}')
        
        # Simulate removing console access
        actions_taken.append('Removed console access')
        
        # Simulate attaching deny-all policy
        actions_taken.append('Attached deny-all policy')
        
        action = IRAction(
            action_id=action_id,
            action_type='revoke_iam_credentials',
            target=user_name,
            status=ActionStatus.COMPLETED,
            timestamp=datetime.now(),
            details={
                'actions_taken': actions_taken,
                'deny_policy_name': 'IR_DenyAll'
            },
            rollback_info={
                'original_keys': ['AKIA1234567890ABCDEF', 'AKIA0987654321FEDCBA']
            }
        )
        
        self.actions_log.append(action)
        return action
    
    def invalidate_role_sessions(self, role_name: str) -> IRAction:
        """Invalidate all active sessions for an IAM role."""
        action_id = str(uuid.uuid4())[:8]
        
        # Simulate adding policy to invalidate sessions
        cutoff_time = datetime.now()
        
        action = IRAction(
            action_id=action_id,
            action_type='invalidate_role_sessions',
            target=role_name,
            status=ActionStatus.COMPLETED,
            timestamp=datetime.now(),
            details={
                'policy_name': 'IR_InvalidateSessions',
                'cutoff_time': cutoff_time.isoformat(),
                'effect': 'All sessions issued before cutoff time are now denied'
            }
        )
        
        self.actions_log.append(action)
        return action

print("IAM Containment ready!")

In [ ]:
# Demo: IAM credential revocation
print("IAM CREDENTIAL REVOCATION DEMO")
print("=" * 50)

iam_containment = IAMContainment()

# Revoke user credentials
print("\n1. Revoking user credentials...")
revoke_action = iam_containment.revoke_user_credentials(
    user_name='compromised_user'
)

print(f"   Action ID: {revoke_action.action_id}")
print(f"   Status: {revoke_action.status.value}")
print(f"   Actions Taken:")
for action in revoke_action.details['actions_taken']:
    print(f"     - {action}")

# Invalidate role sessions
print("\n2. Invalidating role sessions...")
invalidate_action = iam_containment.invalidate_role_sessions(
    role_name='suspicious_lambda_role'
)

print(f"   Action ID: {invalidate_action.action_id}")
print(f"   Status: {invalidate_action.status.value}")
print(f"   Cutoff Time: {invalidate_action.details['cutoff_time']}")

## Part 3: Evidence Collection Automation

Automate the collection and preservation of forensic evidence.

In [ ]:
@dataclass
class EvidenceItem:
    """Represents a piece of collected evidence."""
    item_id: str
    evidence_type: str
    description: str
    s3_path: str
    hashes: Dict[str, str]
    collected_at: datetime
    collected_by: str
    metadata: Dict = field(default_factory=dict)


class EvidenceCollector:
    """Automate evidence collection and chain of custody."""
    
    def __init__(self, case_id: str, s3_bucket: str):
        self.case_id = case_id
        self.s3_bucket = s3_bucket
        self.evidence_items = []
        self.custody_chain = []
    
    def collect_cloudtrail_logs(
        self,
        time_range: Dict[str, datetime]
    ) -> EvidenceItem:
        """Collect CloudTrail logs for investigation."""
        item_id = str(uuid.uuid4())[:8]
        
        # Simulated log collection
        log_content = json.dumps({
            'time_range': {
                'start': time_range['start'].isoformat(),
                'end': time_range['end'].isoformat()
            },
            'event_count': 1234,
            'events': ['... simulated events ...']
        })
        
        # Calculate hashes
        hashes = {
            'md5': hashlib.md5(log_content.encode()).hexdigest(),
            'sha256': hashlib.sha256(log_content.encode()).hexdigest()
        }
        
        s3_path = f's3://{self.s3_bucket}/cases/{self.case_id}/cloudtrail/events.json'
        
        evidence = EvidenceItem(
            item_id=item_id,
            evidence_type='cloudtrail_logs',
            description=f'CloudTrail logs from {time_range["start"]} to {time_range["end"]}',
            s3_path=s3_path,
            hashes=hashes,
            collected_at=datetime.now(),
            collected_by='ir-automation-system',
            metadata={
                'event_count': 1234,
                'time_range': time_range
            }
        )
        
        self.evidence_items.append(evidence)
        self._add_custody_entry('collected', evidence.item_id, 'Initial collection')
        
        return evidence
    
    def collect_vpc_flow_logs(
        self,
        time_range: Dict[str, datetime],
        vpc_id: str
    ) -> EvidenceItem:
        """Collect VPC flow logs for investigation."""
        item_id = str(uuid.uuid4())[:8]
        
        # Simulated flow log collection
        flow_content = json.dumps({
            'vpc_id': vpc_id,
            'record_count': 5678,
            'flows': ['... simulated flows ...']
        })
        
        hashes = {
            'md5': hashlib.md5(flow_content.encode()).hexdigest(),
            'sha256': hashlib.sha256(flow_content.encode()).hexdigest()
        }
        
        s3_path = f's3://{self.s3_bucket}/cases/{self.case_id}/vpc-flow-logs/{vpc_id}.json'
        
        evidence = EvidenceItem(
            item_id=item_id,
            evidence_type='vpc_flow_logs',
            description=f'VPC flow logs for {vpc_id}',
            s3_path=s3_path,
            hashes=hashes,
            collected_at=datetime.now(),
            collected_by='ir-automation-system',
            metadata={
                'vpc_id': vpc_id,
                'record_count': 5678
            }
        )
        
        self.evidence_items.append(evidence)
        self._add_custody_entry('collected', evidence.item_id, 'Initial collection')
        
        return evidence
    
    def _add_custody_entry(self, action: str, item_id: str, notes: str):
        """Add entry to chain of custody."""
        entry = {
            'timestamp': datetime.now().isoformat(),
            'action': action,
            'item_id': item_id,
            'actor': 'ir-automation-system',
            'notes': notes
        }
        self.custody_chain.append(entry)
    
    def generate_chain_of_custody(self) -> Dict:
        """Generate complete chain of custody record."""
        return {
            'case_id': self.case_id,
            'created_at': datetime.now().isoformat(),
            'evidence_items': [
                {
                    'item_id': e.item_id,
                    'type': e.evidence_type,
                    'description': e.description,
                    's3_path': e.s3_path,
                    'hashes': e.hashes,
                    'collected_at': e.collected_at.isoformat()
                }
                for e in self.evidence_items
            ],
            'custody_chain': self.custody_chain
        }

print("Evidence Collector ready!")

In [ ]:
# Demo: Evidence collection
print("EVIDENCE COLLECTION DEMO")
print("=" * 50)

collector = EvidenceCollector(
    case_id='CASE-2024-001',
    s3_bucket='forensics-evidence-bucket'
)

# Set time range for evidence collection
time_range = {
    'start': datetime.now() - timedelta(hours=24),
    'end': datetime.now()
}

# Collect CloudTrail logs
print("\n1. Collecting CloudTrail logs...")
ct_evidence = collector.collect_cloudtrail_logs(time_range)
print(f"   Item ID: {ct_evidence.item_id}")
print(f"   S3 Path: {ct_evidence.s3_path}")
print(f"   SHA256: {ct_evidence.hashes['sha256'][:32]}...")

# Collect VPC flow logs
print("\n2. Collecting VPC flow logs...")
flow_evidence = collector.collect_vpc_flow_logs(time_range, 'vpc-0123456789abcdef0')
print(f"   Item ID: {flow_evidence.item_id}")
print(f"   S3 Path: {flow_evidence.s3_path}")
print(f"   SHA256: {flow_evidence.hashes['sha256'][:32]}...")

# Generate chain of custody
print("\n3. Generating chain of custody...")
custody = collector.generate_chain_of_custody()
print(f"   Case ID: {custody['case_id']}")
print(f"   Evidence Items: {len(custody['evidence_items'])}")
print(f"   Custody Entries: {len(custody['custody_chain'])}")

## Part 4: IR Orchestration

Coordinate multiple containment and evidence collection actions into automated workflows.

In [ ]:
class SeverityLevel(Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"


@dataclass
class IncidentCase:
    """Represents an incident case."""
    case_id: str
    severity: SeverityLevel
    alert_type: str
    affected_resources: List[str]
    created_at: datetime
    status: str = "open"
    actions: List[IRAction] = field(default_factory=list)
    evidence: List[EvidenceItem] = field(default_factory=list)


class IROrchestrator:
    """Orchestrate incident response workflows."""
    
    def __init__(self, evidence_bucket: str):
        self.evidence_bucket = evidence_bucket
        self.ec2_containment = EC2Containment()
        self.iam_containment = IAMContainment()
        self.cases = {}
    
    def handle_alert(self, alert: Dict) -> IncidentCase:
        """Handle an incoming security alert."""
        # Create incident case
        case_id = f"CASE-{datetime.now().strftime('%Y%m%d')}-{uuid.uuid4().hex[:4].upper()}"
        severity = self._determine_severity(alert)
        
        case = IncidentCase(
            case_id=case_id,
            severity=severity,
            alert_type=alert.get('type', 'unknown'),
            affected_resources=alert.get('resources', []),
            created_at=datetime.now()
        )
        
        # Execute response based on severity
        if severity in [SeverityLevel.CRITICAL, SeverityLevel.HIGH]:
            self._execute_critical_response(case, alert)
        elif severity == SeverityLevel.MEDIUM:
            self._execute_medium_response(case, alert)
        else:
            self._execute_low_response(case, alert)
        
        self.cases[case_id] = case
        return case
    
    def _determine_severity(self, alert: Dict) -> SeverityLevel:
        """Determine severity based on alert properties."""
        severity_score = alert.get('severity_score', 0)
        
        if severity_score >= 9:
            return SeverityLevel.CRITICAL
        elif severity_score >= 7:
            return SeverityLevel.HIGH
        elif severity_score >= 4:
            return SeverityLevel.MEDIUM
        else:
            return SeverityLevel.LOW
    
    def _execute_critical_response(self, case: IncidentCase, alert: Dict):
        """Execute critical severity response - immediate containment."""
        # Isolate affected EC2 instances
        for resource in case.affected_resources:
            if resource.startswith('i-'):  # EC2 instance
                action = self.ec2_containment.isolate_instance(
                    instance_id=resource,
                    isolation_sg_id='sg-isolation-critical',
                    reason=f'Critical alert: {alert.get("type")}'
                )
                case.actions.append(action)
                
                # Create forensic snapshot
                snapshot_action = self.ec2_containment.create_forensic_snapshot(
                    instance_id=resource,
                    case_id=case.case_id
                )
                case.actions.append(snapshot_action)
        
        # Revoke compromised credentials
        if 'user' in alert:
            action = self.iam_containment.revoke_user_credentials(
                user_name=alert['user']
            )
            case.actions.append(action)
        
        # Collect evidence
        collector = EvidenceCollector(case.case_id, self.evidence_bucket)
        time_range = {
            'start': datetime.now() - timedelta(hours=48),
            'end': datetime.now()
        }
        
        ct_evidence = collector.collect_cloudtrail_logs(time_range)
        case.evidence.append(ct_evidence)
    
    def _execute_medium_response(self, case: IncidentCase, alert: Dict):
        """Execute medium severity response - containment with review."""
        # Similar to critical but with notification for human review
        for resource in case.affected_resources:
            if resource.startswith('i-'):
                action = self.ec2_containment.create_forensic_snapshot(
                    instance_id=resource,
                    case_id=case.case_id
                )
                case.actions.append(action)
        
        # Collect evidence for review
        collector = EvidenceCollector(case.case_id, self.evidence_bucket)
        time_range = {
            'start': datetime.now() - timedelta(hours=24),
            'end': datetime.now()
        }
        
        ct_evidence = collector.collect_cloudtrail_logs(time_range)
        case.evidence.append(ct_evidence)
    
    def _execute_low_response(self, case: IncidentCase, alert: Dict):
        """Execute low severity response - logging only."""
        # Just log for later review
        pass
    
    def get_case_summary(self, case_id: str) -> Dict:
        """Get summary of a case."""
        case = self.cases.get(case_id)
        if not case:
            return {'error': 'Case not found'}
        
        return {
            'case_id': case.case_id,
            'severity': case.severity.value,
            'alert_type': case.alert_type,
            'status': case.status,
            'affected_resources': case.affected_resources,
            'actions_taken': len(case.actions),
            'evidence_collected': len(case.evidence),
            'created_at': case.created_at.isoformat()
        }

print("IR Orchestrator ready!")

In [ ]:
# Demo: Full IR orchestration
print("IR ORCHESTRATION DEMO")
print("=" * 50)

orchestrator = IROrchestrator(evidence_bucket='forensics-evidence-bucket')

# Simulate a critical security alert
critical_alert = {
    'type': 'cryptocurrency_mining',
    'severity_score': 9.5,
    'resources': ['i-0123456789abcdef0', 'i-0987654321fedcba0'],
    'user': 'compromised_developer',
    'source_ip': '192.168.1.100',
    'timestamp': datetime.now().isoformat()
}

print("\nProcessing Critical Alert...")
print(f"Alert Type: {critical_alert['type']}")
print(f"Severity Score: {critical_alert['severity_score']}")
print(f"Affected Resources: {critical_alert['resources']}")

# Handle the alert
case = orchestrator.handle_alert(critical_alert)

print("\n" + "-" * 50)
print("INCIDENT CASE CREATED")
print("-" * 50)

summary = orchestrator.get_case_summary(case.case_id)
print(f"\nCase ID: {summary['case_id']}")
print(f"Severity: {summary['severity'].upper()}")
print(f"Status: {summary['status']}")
print(f"Actions Taken: {summary['actions_taken']}")
print(f"Evidence Collected: {summary['evidence_collected']}")

print("\nActions Performed:")
for i, action in enumerate(case.actions, 1):
    print(f"  {i}. {action.action_type} on {action.target} - {action.status.value}")

## Part 5: Network Containment

Automate network-level containment actions.

In [ ]:
class NetworkContainment:
    """Network-level containment automation."""
    
    def __init__(self):
        self.blocked_ips = set()
        self.actions_log = []
    
    def block_ip_address(self, ip_address: str, reason: str) -> IRAction:
        """Block an IP address using WAF and Network ACL."""
        action_id = str(uuid.uuid4())[:8]
        actions_taken = []
        
        # Simulate WAF update
        ip_cidr = f'{ip_address}/32'
        actions_taken.append(f'Added {ip_cidr} to WAF block list')
        
        # Simulate NACL update
        rule_number = 100 + len(self.blocked_ips)
        actions_taken.append(f'Added NACL rule {rule_number} to block {ip_address}')
        
        self.blocked_ips.add(ip_address)
        
        action = IRAction(
            action_id=action_id,
            action_type='block_ip',
            target=ip_address,
            status=ActionStatus.COMPLETED,
            timestamp=datetime.now(),
            details={
                'reason': reason,
                'actions_taken': actions_taken,
                'waf_ip_set': 'IR-BlockedIPs',
                'nacl_rule': rule_number
            },
            rollback_info={
                'ip_cidr': ip_cidr,
                'nacl_rule_number': rule_number
            }
        )
        
        self.actions_log.append(action)
        return action
    
    def get_blocked_ips(self) -> List[str]:
        """Get list of currently blocked IPs."""
        return list(self.blocked_ips)

print("Network Containment ready!")

In [ ]:
# Demo: Network containment
print("NETWORK CONTAINMENT DEMO")
print("=" * 50)

network_containment = NetworkContainment()

# Block malicious IPs
malicious_ips = [
    ('192.168.1.100', 'GuardDuty: Cryptocurrency mining C2'),
    ('10.0.0.50', 'CloudTrail: Unauthorized API access'),
    ('172.16.0.25', 'VPC Flow Logs: Data exfiltration attempt')
]

for ip, reason in malicious_ips:
    print(f"\nBlocking IP: {ip}")
    action = network_containment.block_ip_address(ip, reason)
    print(f"  Action ID: {action.action_id}")
    print(f"  Status: {action.status.value}")
    for taken in action.details['actions_taken']:
        print(f"    - {taken}")

print(f"\nTotal Blocked IPs: {len(network_containment.get_blocked_ips())}")

## Part 6: Testing and Validation

Test IR automation functions to ensure reliability.

In [ ]:
class IRAutomationTester:
    """Test IR automation functions."""
    
    def __init__(self):
        self.test_results = []
    
    def test_ec2_isolation(self) -> Dict:
        """Test EC2 isolation function."""
        containment = EC2Containment()
        
        result = containment.isolate_instance(
            instance_id='i-test-instance',
            isolation_sg_id='sg-test-isolation',
            reason='Test isolation'
        )
        
        test = {
            'test_name': 'EC2 Isolation',
            'passed': result.status == ActionStatus.COMPLETED,
            'details': {
                'action_id': result.action_id,
                'status': result.status.value,
                'has_rollback_info': bool(result.rollback_info)
            }
        }
        
        self.test_results.append(test)
        return test
    
    def test_iam_revocation(self) -> Dict:
        """Test IAM credential revocation."""
        containment = IAMContainment()
        
        result = containment.revoke_user_credentials(
            user_name='test-user'
        )
        
        test = {
            'test_name': 'IAM Revocation',
            'passed': (
                result.status == ActionStatus.COMPLETED and
                len(result.details['actions_taken']) > 0
            ),
            'details': {
                'action_id': result.action_id,
                'actions_taken': len(result.details['actions_taken'])
            }
        }
        
        self.test_results.append(test)
        return test
    
    def test_evidence_collection(self) -> Dict:
        """Test evidence collection."""
        collector = EvidenceCollector('TEST-CASE', 'test-bucket')
        
        time_range = {
            'start': datetime.now() - timedelta(hours=1),
            'end': datetime.now()
        }
        
        evidence = collector.collect_cloudtrail_logs(time_range)
        
        test = {
            'test_name': 'Evidence Collection',
            'passed': (
                evidence.item_id is not None and
                'sha256' in evidence.hashes
            ),
            'details': {
                'item_id': evidence.item_id,
                'has_hash': 'sha256' in evidence.hashes,
                's3_path': evidence.s3_path
            }
        }
        
        self.test_results.append(test)
        return test
    
    def run_all_tests(self) -> Dict:
        """Run all tests and return summary."""
        self.test_ec2_isolation()
        self.test_iam_revocation()
        self.test_evidence_collection()
        
        passed = sum(1 for t in self.test_results if t['passed'])
        failed = len(self.test_results) - passed
        
        return {
            'total_tests': len(self.test_results),
            'passed': passed,
            'failed': failed,
            'success_rate': passed / len(self.test_results) if self.test_results else 0,
            'results': self.test_results
        }

print("IR Automation Tester ready!")

In [ ]:
# Demo: Run IR automation tests
print("IR AUTOMATION TESTING")
print("=" * 50)

tester = IRAutomationTester()
results = tester.run_all_tests()

print(f"\nTest Results:")
print("-" * 50)

for test in results['results']:
    status = "PASS" if test['passed'] else "FAIL"
    print(f"  [{status}] {test['test_name']}")

print("\n" + "-" * 50)
print("SUMMARY")
print(f"  Total Tests: {results['total_tests']}")
print(f"  Passed: {results['passed']}")
print(f"  Failed: {results['failed']}")
print(f"  Success Rate: {results['success_rate']:.0%}")

## Key Takeaways

1. **Containment Automation** - Automate EC2 isolation, IAM revocation, and network blocking
2. **Evidence Collection** - Automated CloudTrail and VPC flow log collection with hashing
3. **Chain of Custody** - Maintain forensically sound evidence handling
4. **Orchestration** - Coordinate multiple actions based on alert severity
5. **Testing** - Validate IR automation before production deployment

## IR Automation Checklist

| Phase | Automated Actions |
|-------|------------------|
| Detection | Alert correlation, severity classification |
| Containment | Instance isolation, credential revocation, IP blocking |
| Evidence | Disk snapshots, log collection, hashing |
| Documentation | Case creation, chain of custody, action logging |
| Notification | Team alerts, escalation |

## Next Steps

- **Lab 49**: LLM Red Teaming - Test AI security
- **Lab 21**: Threat Intelligence - Enrich incidents with TI
- **Lab 22**: SOAR Integration - Connect to orchestration platforms